In [1]:
import numpy as np
import utils
from utils import loadmat
from extractDigitFeatures import extractDigitFeatures
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Conv3D, Flatten, MaxPool2D, AveragePooling2D, BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.python.keras import metrics
from keras import regularizers
from keras.constraints import unit_norm
from sklearn.model_selection import StratifiedKFold

Using TensorFlow backend.


In [2]:
def reformat(x):
    new_x = np.zeros((x.shape[2], x.shape[0], x.shape[1], 1))
    for i in range(x.shape[2]):
        new_x[i, :] = x[:, :, i][:, :, np.newaxis]
        return new_x

In [9]:
files[0]

'digits-jitter.mat'

In [3]:
files = ['digits-jitter.mat', 'digits-normal.mat', 'digits-scaled.mat']
data = utils.loadmat('../data/{}'.format(files[0]))
trainSet, testSet, valSet = 1, 2, 3

x_train = reformat(data['x'][:, :, data['set']==trainSet ])
y_train = (data['y'][data['set']==trainSet])
x_val = reformat(data['x'][:, :, data['set']==valSet])
y_val = (data['y'][data['set']==valSet])
x_test = reformat(data['x'][:, :, data['set']==testSet])
y_test = (data['y'][data['set']==testSet])

X = np.vstack([x_train, x_val])
Y = np.vstack([y_train[:, np.newaxis], y_val[:, np.newaxis]])

In [4]:
def model_extractor(activation_func='relu', weight_decay=1e-4):
    # Creating an AlexNet Classifier
    model = Sequential()

    #Instantiating Layer 1
    model.add(Conv2D(48, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='valid', 
                    kernel_constraint=unit_norm(), kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))

    # #Instantiating Layer 2
    model.add(Conv2D(96, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='same', 
                    kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(BatchNormalization())
    # #Instantiating Layer 3
    model.add(Conv2D(192, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='same',
                    kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(BatchNormalization())

    # #Instantiating Layer 4
    model.add(Conv2D(192, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='same', 
                    kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(BatchNormalization())

    # #Instantiating Layer 5
    model.add(Conv2D(256, kernel_size=(3, 3), strides=(1, 1), activation=activation_func, padding='same', 
                    kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(BatchNormalization())
    model.add(Flatten())

    #Instantiating Layer 6
    model.add(Dense(512, activation=activation_func)) 

    # #Instantiating Layer 8
    model.add(Dense(256, activation=activation_func))

    #Output Layer
    model.add(Dense(10, activation='softmax'))
    return model

In [7]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
best_model, best_accuracy = -1, 0
for train, test in kfold.split(X, Y):
    model = model_extractor('selu')
    model.compile(loss="categorical_crossentropy", optimizer='Adam', validation_data=(x_val, y_val), metrics=["accuracy"])
    model.fit(X[train], Y[train], epochs=15, batch_size=200, verbose=0)
    scores = model.evaluate(X[test], Y[test], batch_size=int(y_val.size/5), verbose=0)
    print ("Accuracy = {}".format(scores[1]))
    
    if scores[1] > best_accuracy:
        best_accuracy = scores[1]
        best_model = model

Accuracy = 0.0
Accuracy = 1.0
Accuracy = 0.0


KeyboardInterrupt: 

In [8]:
best_model.evaluate(x_test, y_test)

500/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[87259.70597807599, 0.998]